In [1]:
from time import sleep
from onvif import ONVIFCamera
from pynput import keyboard

In [2]:
class Camera(object):
    XMAX = 1
    XMIN = -1
    YMAX = 1
    YMIN = -1
    MAXFOCUS = 7
    MINFOCUS = -7
    
    def __init__(self, *args, **kwargs):
        self.camera = ONVIFCamera(*args, **kwargs)
        self.media = self.camera.create_media_service()
        self.imaging = self.camera.create_imaging_service()
        self.ptz = self.camera.create_ptz_service()

        self.media_profile = self.media.GetProfiles()[0]
        self.vstoken = self.media.GetVideoSources()[0]._token
        
        request = self.ptz.create_type('GetConfigurationOptions')
        request.ConfigurationToken = self.media_profile.PTZConfiguration._token
        ptz_configuration_options = self.ptz.GetConfigurationOptions(request)
        self.XMAX = ptz_configuration_options.Spaces.ContinuousPanTiltVelocitySpace[0].XRange.Max
        self.XMIN = ptz_configuration_options.Spaces.ContinuousPanTiltVelocitySpace[0].XRange.Min
        self.YMAX = ptz_configuration_options.Spaces.ContinuousPanTiltVelocitySpace[0].YRange.Max
        self.YMIN = ptz_configuration_options.Spaces.ContinuousPanTiltVelocitySpace[0].YRange.Min

    # получаем координаты камеры
    def get_status(self):
        request = self.ptz.create_type('GetStatus')
        request.ProfileToken = self.media_profile._token
        return self.ptz.GetStatus(request)
    
    # получаем данные по фокусу
    def get_focus_options(self):
        request = self.imaging.create_type('GetMoveOptions')
        request.VideoSourceToken = self.vstoken
        return self.imaging.GetMoveOptions(request)
    
    # переход по абсолютным координатам
    def absolute_move(self, x=0, y=0, z=0, velocity=1e-3):
        request = self.ptz.create_type('AbsoluteMove')
        request.ProfileToken = self.media_profile._token
        request.Position.PanTilt._x = x
        request.Position.PanTilt._y = y
        request.Position.Zoom._x = z
        self.ptz.AbsoluteMove(request)
    
    # получаем координаты камеры
    def go_to_focus_position(self, x):
        request = self.imaging.create_type('Move')
        request.Focus = {}
        request.VideoSourceToken = self.vstoken
        request.Absolute = {'Position': x}
        self.imaging.Move(request)
    
    # перемещение камеры в Continuous move
    def perform_move(self, ptz, request, timeout):
        ptz.ContinuousMove(request)
        sleep(timeout)
        ptz.Stop({'ProfileToken': request.ProfileToken})
    
    # режим ручной фокусировки
    def set_manual_focus(self):
        name = "SetImagingSettings"
        self.imaging.create_type(name)
        request = {'VideoSourceToken': self.vstoken, 
                   'ImagingSettings': {'Focus': {'AutoFocusMode': 'MANUAL'}}
                  }
        self.imaging.SetImagingSettings(request)
    
    # режим автоматической фокусировки
    def reset_manual_focus(self):
        name = "SetImagingSettings"
        self.imaging.create_type(name)
        request = {'VideoSourceToken': self.vstoken, 
                   'ImagingSettings': {'Focus': {'AutoFocusMode': 'AUTO'}}
                  }
        self.imaging.SetImagingSettings(request)
    
    # настройка фокуса
    def perform_focus_move(self, request):
        self.imaging.Move(request)
        self.imaging.Stop({'VideoSourceToken': self.vstoken})
    
    # увеличение изображения на камере
    def zoom_plus(self, ptq, request, timeout=1):
        request.Velocity.Zoom._x = 1./5
        self.perform_move(self.ptz, request, timeout)
    
    # уменьшение изображения на камере
    def zoom_minus(self, ptq, request, timeout=1):
        request.Velocity.Zoom._x = -1./5
        self.perform_move(self.ptz, request, timeout)
    
    # поворот камеры вверх
    def move_up(self, ptz, request, timeout=1):
        request.Velocity.PanTilt._x = 0
        request.Velocity.PanTilt._y = self.YMAX / 2
        self.perform_move(ptz, request, timeout)

    # поворот камеры вниз
    def move_down(self, ptz, request, timeout=1):
        request.Velocity.PanTilt._x = 0
        request.Velocity.PanTilt._y = self.YMIN / 2
        self.perform_move(ptz, request, timeout)

    # поворот камеры направо
    def move_right(self, ptz, request, timeout=1):
        request.Velocity.PanTilt._x = self.XMAX / 2
        request.Velocity.PanTilt._y = 0
        self.perform_move(ptz, request, timeout)

    # поворот камеры налево
    def move_left(self, ptz, request, timeout=1):
        request.Velocity.PanTilt._x = self.XMIN / 2
        request.Velocity.PanTilt._y = 0
        self.perform_move(ptz, request, timeout)
    
    # поворот камеры во все четыре стороны в режиме Continuous move
    def continuous_move(self):
        request = self.ptz.create_type('ContinuousMove')
        request.ProfileToken = self.media_profile._token
        
        # вправо
        self.move_right(self.ptz, request)
        # влево
        self.move_left(self.ptz, request)
        # вверх
        self.move_up(self.ptz, request)
        # вниз
        self.move_down(self.ptz, request)
    
    # фокусировка
    def focus_in(self, request):
        request.Focus = {'Continuous': {'Speed': self.MAXFOCUS / 2}}
        self.perform_focus_move(request)
    
    # расфокусировка
    def focus_out(self, request):
        request.Focus = {'Continuous': {'Speed': self.MINFOCUS / 2}}
        self.perform_focus_move(request)


Подключение к камере

In [4]:
mycam = Camera(
    'address',
    port,
    'user',
    'password',
    '/etc/onvif/wsdl/'
)

Вывод координат камеры 

In [5]:
print mycam.get_status()


(PTZStatus){
   Position = 
      (PTZVector){
         PanTilt = 
            (Vector2D){
               _y = 1.0
               _x = 0.994444
               _space = "http://www.onvif.org/ver10/tptz/PanTiltSpaces/PositionGenericSpace"
            }
         Zoom = 
            (Vector1D){
               _x = 0.0
               _space = "http://www.onvif.org/ver10/tptz/ZoomSpaces/PositionGenericSpace"
            }
      }
   Error = "NO error"
   UtcTime = 2019-03-02 21:41:21
 }


Переход по абсолютным координатам

In [6]:
mycam.absolute_move(0.8, -0.3, 0)

In [7]:
mycam.absolute_move(-0.6, 0.5, 0.7)

Перемещение камеры в Continuous move

In [8]:
mycam.continuous_move()

In [9]:
print mycam.get_focus_options()

(MoveOptions20){
   Continuous = 
      (ContinuousFocusOptions){
         Speed = 
            (FloatRange){
               Min = -7.0
               Max = 7.0
            }
      }
 }


Установка режима ручной фокусировки

In [10]:
mycam.set_manual_focus()

Изменение фокуса 

In [11]:
req = mycam.imaging.create_type('Move')
req.VideoSourceToken = mycam.vstoken
req.Focus = {'Continuous': {'Speed': 7}}
mycam.perform_focus_move(req)
mycam.perform_focus_move(req)
mycam.perform_focus_move(req)
mycam.perform_focus_move(req)

# Работа с камерой с помощью клавиатуры

In [12]:
class KeyboardCam(Camera):
    def __init__(self, *args, **kwargs):
        super(KeyboardCam, self).__init__(*args, **kwargs)
        
    def on_release(self, key):
        if key == keyboard.Key.esc:
            return False
    
    def on_press(self, key):
        
        # зум - стрелки вверх/вниз, движение камеры - стрелки влево/вправо
        self.request = self.ptz.create_type('ContinuousMove')
        self.request.ProfileToken = self.media_profile._token
        
        if key == keyboard.Key.up:
                self.zoom_plus(self.ptz, self.request, timeout=0)
        if key == keyboard.Key.down:
                self.zoom_minus(self.ptz, self.request, timeout=0)
        if key == keyboard.Key.left:
            self.move_left(self.ptz, self.request, timeout=0)
        if key == keyboard.Key.right:
            self.move_right(self.ptz, self.request, timeout=0)
        
        # настройка фокуса - клавиши i/o
        request = self.imaging.create_type('Move')
        request.VideoSourceToken = self.vstoken
        try:
            if key.char == "i":
                self.focus_in(request)
        except AttributeError:
            pass
        try:
            if key.char == "o":
                self.focus_out(request)
        except AttributeError:
            pass
         
    def listen_input(self):
        with keyboard.Listener(
            on_press=self.on_press,
            on_release=self.on_release
        ) as listener:
            listener.join()

In [13]:
keyboard_cam = KeyboardCam(
    'address',
    port,
    'user',
    'password',
    '/etc/onvif/wsdl/'
)

In [14]:
keyboard_cam.listen_input()